In [1]:
import os

In [2]:
%pwd

'c:\\Users\\karth\\ML_practice\\Projects\\EndToEnd_MLOps_Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\karth\\ML_practice\\Projects\\EndToEnd_MLOps_Project'

In [5]:
import pandas as pd

data = pd.read_csv("artifacts/data_ingestion/Car_Selling_Prices_data.csv")
data.head()

,brand,year,km_driven,fuel,seller_type,transmission,owner,selling_price
0,Maruti,2007,70000,Petrol,Individual,Manual,First Owner,60000
1,Maruti,2007,50000,Petrol,Individual,Manual,First Owner,135000
2,Hyundai,2012,100000,Diesel,Individual,Manual,First Owner,600000
3,Datsun,2017,46000,Petrol,Individual,Manual,First Owner,250000
4,Honda,2014,141000,Diesel,Individual,Manual,Second Owner,450000


In [6]:
data.shape

(4340, 8)

In [7]:
data.columns

Index(['brand', 'year', 'km_driven', 'fuel', 'seller_type', 'transmission',
       'owner', 'selling_price'],
      dtype='object')

In [8]:
data.isnull().sum()

brand            0
year             0
km_driven        0
fuel             0
seller_type      0
transmission     0
owner            0
selling_price    0
dtype: int64

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4340 entries, 0 to 4339
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   brand          4340 non-null   object
 1   year           4340 non-null   int64 
 2   km_driven      4340 non-null   int64 
 3   fuel           4340 non-null   object
 4   seller_type    4340 non-null   object
 5   transmission   4340 non-null   object
 6   owner          4340 non-null   object
 7   selling_price  4340 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 271.4+ KB


In [10]:
data["brand"].nunique()

29

In [11]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [12]:
from src.ML_Project.constants import *
from src.ML_Project.utils.common import read_yaml, create_directories

In [13]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
        ):
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            self.schema = read_yaml(schema_filepath)
            create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
          config = self.config.data_validation
          schema = self.schema.COLUMNS
          create_directories([config.root_dir])         

          data_validation_config = DataValidationConfig(
                root_dir= config.root_dir,
                STATUS_FILE= config.STATUS_FILE,
                unzip_data_dir= config.unzip_data_dir,
                all_schema = schema
          )
          return data_validation_config   

In [14]:
from src.ML_Project import logger

In [15]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    def validate_all_columns(self) -> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)
            all_schema = self.config.all_schema.keys()

            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation Status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation Status: {validation_status}")
            return validation_status

        except Exception as e:
            raise e

In [17]:

try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2025-03-23 14:53:16,055: INFO: common: Yaml file:config\config.yaml loaded successfully]
[2025-03-23 14:53:16,064: INFO: common: Yaml file:params.yaml loaded successfully]
[2025-03-23 14:53:16,071: INFO: common: Yaml file:schema.yaml loaded successfully]
[2025-03-23 14:53:16,074: INFO: common: created directory at: artifacts]
[2025-03-23 14:53:16,077: INFO: common: created directory at: artifacts/data_validation]
